In [ ]:
import pandas as pd
from tweepy.client import Client
import tweepy
import os
import time
from dateutil import parser

bearer_token = "FILL IN"
API_Key = "FILL IN"
API_secret_key = "FILL IN"
client = Client(bearer_token = bearer_token, 
                consumer_key = API_Key,
                consumer_secret = API_secret_key,
                wait_on_rate_limit = False,
               return_type = dict)

# check if the file exists in directory
def check_file(filename):
    if os.path.exists(filename):
        #print("file found to update:", filename)
        return True
    else:
        #print("no file found, creating new file:", filename)
        return False

# merge old and new data and save as csv
def merge_data(new_data, old_data, filename):
    merged_data = pd.concat([old_data, new_data])
    merged_data.to_csv(filename, sep = "|")
    return merged_data


**SCRAPE TWEETS BEFORE AND AFTER INVASION**

In [ ]:
# start timer
start_time = time.time()

# initiate start and end times
start_before_invasion = '2022-02-23T02:30:00Z'
invasion_time = '2022-02-24T02:30:00Z'
start_time = '2022-02-24T03:00:00Z'
end_after_invasion = '2022-02-25T02:30:00Z'

for item in ['before', 'after']:
    
    if item == 'before':
        print("Scraping tweets from before the invasion")
        
        # initiate start and end time
        start_time = start_before_invasion
        end_time = invasion_time
        
        # initiate filenames
        filename_tweets = 'before_invasion_tweets.csv'
        filename_users = 'before_invasion_users.csv'
        filename_places = 'before_invasion_places.csv'
        
    if item == 'after':
        print("Scraping tweets from after the invasion")
        
        # initiate start and end time
        start_time = start_time
        end_time = end_after_invasion
        
        # initiate filenames
        filename_tweets = 'after_invasion_tweets.csv'
        filename_users = 'after_invasion_users.csv'
        filename_places = 'after_invasion_places.csv'
        
    tweets = client.search_all_tweets(query = '(Ukraine OR #Ukraine OR Russia OR #Russia OR Putin OR #Putin OR Zelensky OR #Zelensky OR #UkraineRussianWar) lang:en -is:reply -is:quote -is:retweet', expansions = ["author_id", "geo.place_id"], 
                           user_fields = ["created_at", "public_metrics", "verified"], 
                           tweet_fields = ["author_id", "created_at", "public_metrics", "lang", "entities", "conversation_id", "referenced_tweets"],
                          place_fields = ['country'],
                        start_time = start_time, end_time = end_time, max_results = 500)

    # store dataframes
    old_tweets = pd.json_normalize(tweets['data'])
    old_tweets['text'] = old_tweets.apply(lambda row: row['text'].replace("|", ""), axis = 1) # remove | from tweets

    old_users = pd.json_normalize(tweets['includes']['users'])
    old_users['username'] = old_users.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
    old_users['name'] = old_users.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name

    if "places" in [item for item in tweets['includes']]:
        old_places = pd.json_normalize(tweets['includes']['places'])

    # go through 
    while ('next_token' in tweets['meta']):
        
        # print update
        print("total number of tweets: ", len(old_tweets))

        # sleep for 3 seconds to prevent reaching the rate limit
        time.sleep(3)

        next_token = tweets['meta']['next_token']
        tweets = client.search_all_tweets(query = '(Ukraine OR #Ukraine OR Russia OR #Russia OR Putin OR #Putin OR Zelensky OR #Zelensky OR #UkraineRussianWar) lang:en -is:reply -is:quote -is:retweet', expansions = ["author_id", "geo.place_id"], 
                               user_fields = ["created_at", "public_metrics", "verified"], 
                               tweet_fields = ["author_id", "created_at", "public_metrics", "lang", "entities", "conversation_id", "referenced_tweets"],
                              place_fields = ['country'],
                            start_time = start_time, end_time = end_time, next_token = next_token, max_results = 500)

        # update tweet csv
        new_tweets = pd.json_normalize(tweets['data'])
        new_tweets['text'] = new_tweets.apply(lambda row: row['text'].replace("|", ""), axis = 1) # remove | from tweets
        old_tweets = merge_data(new_tweets, old_tweets, filename_tweets)

        # update users csv
        new_users = pd.json_normalize(tweets['includes']['users'])
        new_users['username'] = new_users.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
        new_users['name'] = new_users.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name
        old_users = merge_data(new_users, old_users, filename_users)

        # check if place
        if "places" in [item for item in tweets['includes']]:
            new_places = pd.json_normalize(tweets['includes']['places'])

            if check_file(filename_places):
                old_places = merge_data(new_places, old_places, filename_places)
            else: 
                old_places = new_places

# end timer
end_time = time.time()
print("time in sec elapsed: ", end_time - start_time)
print("time in min elapsed: ", (end_time - start_time) / 60)

**SCRAPE EXTRA TWEETS BEFORE INVASION**

In [ ]:
# start timer
start_time = time.time()

# initiate start and end times
start_time = '2022-02-21T02:30:00Z'
end_time = '2022-02-23T02:30:00Z'

# initiate filenames
filename_tweets = 'before2_invasion_tweets.csv'
filename_users = 'before2_invasion_users.csv'
filename_places = 'before2_invasion_places.csv'


tweets = client.search_all_tweets(query = '(Ukraine OR #Ukraine OR Russia OR #Russia OR Putin OR #Putin OR Zelensky OR #Zelensky OR #UkraineRussianWar) lang:en -is:reply -is:quote -is:retweet', expansions = ["author_id", "geo.place_id"], 
                       user_fields = ["created_at", "public_metrics", "verified"], 
                       tweet_fields = ["author_id", "created_at", "public_metrics", "lang", "entities", "conversation_id", "referenced_tweets"],
                      place_fields = ['country'],
                    start_time = start_time, end_time = end_time, max_results = 500)

# store dataframes
old_tweets = pd.json_normalize(tweets['data'])
old_tweets['text'] = old_tweets.apply(lambda row: row['text'].replace("|", ""), axis = 1) # remove | from tweets

old_users = pd.json_normalize(tweets['includes']['users'])
old_users['username'] = old_users.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
old_users['name'] = old_users.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name

if "places" in [item for item in tweets['includes']]:
    old_places = pd.json_normalize(tweets['includes']['places'])

# go through 
while (('next_token' in tweets['meta']) and (len(old_tweets) < 900000)):

    # print update
    print("total number of tweets: ", len(old_tweets))

    # sleep for 3 seconds to prevent reaching the rate limit
    time.sleep(3)

    next_token = tweets['meta']['next_token']
    tweets = client.search_all_tweets(query = '(Ukraine OR #Ukraine OR Russia OR #Russia OR Putin OR #Putin OR Zelensky OR #Zelensky OR #UkraineRussianWar) lang:en -is:reply -is:quote -is:retweet', expansions = ["author_id", "geo.place_id"], 
                           user_fields = ["created_at", "public_metrics", "verified"], 
                           tweet_fields = ["author_id", "created_at", "public_metrics", "lang", "entities", "conversation_id", "referenced_tweets"],
                          place_fields = ['country'],
                        start_time = start_time, end_time = end_time, next_token = next_token, max_results = 500)

    # update tweet csv
    new_tweets = pd.json_normalize(tweets['data'])
    new_tweets['text'] = new_tweets.apply(lambda row: row['text'].replace("|", ""), axis = 1) # remove | from tweets
    old_tweets = merge_data(new_tweets, old_tweets, filename_tweets)

    # update users csv
    new_users = pd.json_normalize(tweets['includes']['users'])
    new_users['username'] = new_users.apply(lambda row: row['username'].replace("|", ""), axis = 1) # remove | from username
    new_users['name'] = new_users.apply(lambda row: row['name'].replace("|", ""), axis = 1) # remove | from name
    old_users = merge_data(new_users, old_users, filename_users)

    # check if place
    if "places" in [item for item in tweets['includes']]:
        new_places = pd.json_normalize(tweets['includes']['places'])

        if check_file(filename_places):
            old_places = merge_data(new_places, old_places, filename_places)
        else: 
            old_places = new_places

# end timer
end_time = time.time()
print("time in sec elapsed: ", end_time - start_time)
print("time in min elapsed: ", (end_time - start_time) / 60)